# Data Science Nanodegree - Blog Post
Created By: Renan Vieira Dias

--- 

### Rubrics for the Work
https://review.udacity.com/#!/rubrics/1507/view

### Titles For The Article

 - KeyWords:
    - Data
    - Job
    - Work
    - Employ
    - Skills
    - Language
    - Salary
    - Experience
    - Seniority
  
  
 - Brain Storm:
    - Requirements for Data Jobs
    - Data Jobs tools
    - What Data tells about Data Jobs
    - How much each skill is worth it on data



## Questions
  
- Which skills does each data job require?
    - ['LanguageWorkedWith']
    - ['LanguageDesireNextYear']
    - ['DatabaseWorkedWith']
    - ['DatabaseDesireNextYear']
    - ['PlatformWorkedWith']
    - ['PlatformDesireNextYear']
    - ['WebFrameWorkedWith']
    - ['WebFrameDesireNextYear']
    - ['MiscTechWorkedWith']
    - ['MiscTechDesireNextYear']
    - ['DevEnviron']
    - ['OpSys']
  
  
- How long you have to be in the field to get to be senior? (Or equivalent money)
    - ['YearsCodePro']
    - ['CurrencySymbol','CurrencyDesc','CompTotal','CompFreq','ConvertedComp','WorkWeekHrs']
        - alazar: the salary is different for each profession. (0-25%,26-50%,75%,)

  
- What are some variables that influence your salary per hours?
    - ['EdLevel']
    - ALL of the above ( Explain better this part)
        - alazar: for each of the profession or hole group?? sugestion: only the data professions

## Segment
- ['Country'] = ????? 
    - (Sugestions: Top countries by volume of type)
- ['Employment'] = ['Employed full-time','Employed part-time']
- ['devtype_interest'] = ['Data or business analyst','Data scientist or machine learning specialist','Database administrator','Engineer, data']

### Imports

In [ ]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np

### Functions

In [ ]:
def optionIsInCell(cell, option):
    if str(cell).find(option)!=-1:
        return 1
    return 0

In [ ]:
def transformingColumnByOptions(df, column, options, prefix = False, separator='_'):
    for each in options:
        new_column = column + separator + each if prefix else each
        df[new_column] = df[column].apply(optionIsInCell, option = each)
    return df

In [ ]:
def isOneOfTheList(x, list_options):
    for each in list_options:
        if each in x:
            return True
    return False

In [ ]:
def getOptionOfMultipleSelection(df, column, separator = ';'):
    flat_list = []
    for sublist in df[column].dropna().str.split(pat=separator, expand=False):
        for item in sublist:
            if item not in flat_list:
                flat_list.append(item)
    return flat_list

### Reading Data Files

In [ ]:
df = pd.read_csv('survey_results_public.csv')
df_schema = pd.read_csv('survey_results_schema.csv')

### Collecting Columns Options & Decoding

In [ ]:
columnsOptions = {}
columnsToGetOptions =[
        'DevType'
        ,'EdLevel'
        ,'LanguageWorkedWith'
        ,'LanguageDesireNextYear'
        ,'DatabaseWorkedWith'
        ,'DatabaseDesireNextYear'
        ,'PlatformWorkedWith'
        ,'PlatformDesireNextYear'
        ,'WebFrameWorkedWith'
        ,'WebFrameDesireNextYear'
        ,'MiscTechWorkedWith'
        ,'MiscTechDesireNextYear'
        ,'DevEnviron'
        ,'OpSys'
    ]
for each in columnsToGetOptions:
    tempList = getOptionOfMultipleSelection(df,each)
    tempList.sort()
    columnsOptions[each] = tempList

In [ ]:
# Creating each skill columns
for questionColumn in columnsToGetOptions:
    df = transformingColumnByOptions(df, questionColumn, columnsOptions[questionColumn], separator='_')

### Checking the Employment
- Columns: Employment
- Question: Which of the following best describes your current employment status?
- Interest In the following DevTypes:
    - 'Employed full-time'
    - 'Employed part-time'

In [ ]:
# Get only employed surveys
df = df[ df['Employment'].isin(['Employed full-time','Employed part-time']) ]

### Checking Every Work Title
- Columns: DevType
- Question: Which of the following describe you? Please select all that apply.
- Interest In the following DevTypes:
    - 'Data or business analyst'
    - 'Data scientist or machine learning specialist'
    - 'Database administrator'
    - 'Engineer, data'


In [ ]:
devtype_interest = [
    'Data or business analyst'
    ,'Data scientist or machine learning specialist'
    ,'Database administrator'
    ,'Engineer, data'
]

# Getting a list of the other jobs
devtype_not_interest = []
for each in columnsOptions['DevType']:
    if not each in devtype_interest:
        devtype_not_interest.append(each)

In [ ]:
# Clearing those with no interest on data DevType
df = df.dropna(subset = ['DevType']).copy()
# df['datajobs'] = df['DevType'].apply( isOneOfTheList, list_options = devtype_interest)
# df = df[df['datajobs']]
# df = df.drop(['datajobs'], axis=1)

In [ ]:
dataJobsOutputs = [
    'Data or business analyst'
    ,'Data scientist or machine learning specialist'
    ,'Database administrator'
    ,'Engineer, data'
    ,'Data Misture'
    ,'Data Partialy'
    ,'Not Data'
]

# Creating the data jobs Categories
def dataJobsClassificationMicro(DevType):
    for interest in devtype_interest:
        if interest == DevType:
            return interest
    for interest in devtype_interest:
        if interest in DevType:
            for not_interest in devtype_not_interest:
                if not_interest in DevType:
                    return 'Data Partialy'
            return 'Data Misture'
    return 'Not Data'

def dataJobsClassificationMacro(DevType):
    for interest in devtype_interest:
        if interest == DevType:
            return 'DataJob'
    for interest in devtype_interest:
        if interest in DevType:
            for not_interest in devtype_not_interest:
                if not_interest in DevType:
                    return 'Data Partialy'
            return 'DataJob'
    return 'Not Data'

In [ ]:
df['datajobsCategoryMicro'] = df['DevType'].apply( dataJobsClassificationMicro )
df['datajobsCategoryMacro'] = df['DevType'].apply( dataJobsClassificationMacro )

In [ ]:
df[['DevType','datajobsCategoryMicro']].head(5)

In [ ]:
df['datajobsCategoryMicro'].value_counts()

### Understanding Job Skills

question. 


### Plotting Each Interest

In [ ]:
groupedResults = df.groupby(['datajobsCategoryMacro']).mean()
for skill in columnsOptions:
        groupedResults.reset_index()[columnsOptions[skill]].transpose().plot.barh()
        plt.legend(groupedResults.index)
        plt.title(skill)
        plt.show()


In [ ]:
# .sort_values('Speed')
for eachOutput in dataJobsOutputs:
    groupedResults = df[df['datajobsCategoryMicro']==eachOutput].groupby(['datajobsCategoryMicro']).mean()
    for skill in columnsOptions:
            groupedResults.reset_index()[columnsOptions[skill]].transpose().sort_values(0,ascending=True).plot.barh()
            plt.legend(groupedResults.index)
            plt.title(eachOutput + ' - ' + skill)
            plt.show()


In [ ]:
# Checking the distribution of 'Data Misture'
groupedResults.reset_index()[groupedResults.reset_index()['datajobsCategoryMicro']=='Data Misture'][devtype_interest].transpose()

(Review) C


Sugestion: DATA RELATED AREA (GROUP FIRST?)

### Calculating Equivalent Salary
  
  
To calculate the seniority of a salary?
 - By Country
 - By Job Profession
 - At least 100
 - Histogram with quarter?
     - Junior 0-25%
     - Pleno 26-50%
     - Senior 51-75%
     - Specialist 100%
  
  
 - Plot
     - Top countries 1 for each top 5
     - X - Years of experience
     - Y - DollarPerHour
     - Colour - DataType

In [ ]:
weeksInYear = 52
df['DollarPerHour'] = df['ConvertedComp']/df['WorkWeekHrs']/weeksInYear
df[['Country','CurrencySymbol','CurrencyDesc','CompTotal','CompFreq','ConvertedComp','WorkWeekHrs','DollarPerHour']].head(5)

In [ ]:
def cleaningYearsOfCode(text):
    if text == 'Less than 1 year':
        return 0.5
    return text

In [ ]:
df['YearsCodeProFix'] = df['YearsCodePro'].apply(cleaningYearsOfCode)
df['YearsCodeProFix'].head()

In [ ]:
dfSalary = df[df['Country']=='Brazil']
dfSalaryClean = dfSalary.dropna( subset = ['Country','YearsCodeProFix','DollarPerHour','datajobsCategoryMicro'])
dfSalaryClean[['Country','YearsCodeProFix','DollarPerHour','datajobsCategoryMicro']].head(5)

In [ ]:
dataJobsOutputs

In [ ]:
‘b’	blue
‘g’	green
‘r’	red
‘c’	cyan
‘m’	magenta
‘y’	yellow
‘k’	black

In [ ]:
colors = ['b','g','r','c','m','y','k','b','b','b']

dataPlots = {}
i = 0
# for each in ['Data or business analyst']:
for each in dataJobsOutputs:
    dataPlots[each] = plt.scatter(dfSalaryClean['YearsCodeProFix'], dfSalaryClean['DollarPerHour'], color=colors[i])
    i = i + 1

plt.legend(dataPlots.values(),
           dataPlots.keys(),
           scatterpoints=1,
           loc='lower left',
           ncol=3,
           fontsize=8)


plt.ylim((0, 100))
plt.xlim((0, 20))
plt.title( 'Year Of Coding VS Salary By Country and Job' )

plt.show()

In [ ]:
colors = ['b', 'c', 'y', 'm', 'r']

lo = plt.scatter(np.random.random(10), np.random.random(10), marker='x', color=colors[0])
ll = plt.scatter(np.random.random(10), np.random.random(10), marker='o', color=colors[0])
l  = plt.scatter(np.random.random(10), np.random.random(10), marker='o', color=colors[1])
a  = plt.scatter(np.random.random(10), np.random.random(10), marker='o', color=colors[2])
h  = plt.scatter(np.random.random(10), np.random.random(10), marker='o', color=colors[3])
hh = plt.scatter(np.random.random(10), np.random.random(10), marker='o', color=colors[4])
ho = plt.scatter(np.random.random(10), np.random.random(10), marker='x', color=colors[4])

plt.legend((lo, ll, l, a, h, hh, ho),
           ('Low Outlier', 'LoLo', 'Lo', 'Average', 'Hi', 'HiHi', 'High Outlier'),
           scatterpoints=1,
           loc='lower left',
           ncol=3,
           fontsize=8)

plt.show()

### Years of code

In [ ]:
df['YearsCodePro'].head(50)

In [ ]:
df['Country'].value_counts()